In [1]:
NEO4J_URI = "<your url>"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "<your pwd>"
NEO4J_DATABASE = "neo4j"

In [2]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore


from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

/home/anish/anaconda3/envs/llama_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
documents = SimpleDirectoryReader(input_files = ['data/Solar_System_Exploration_and_India_contributions.pdf']).load_data()
print(len(documents))
print(f"Document Metadata: {documents[0].metadata}")


100
Document Metadata: {'page_label': '1', 'file_name': 'Solar_System_Exploration_and_India_contributions.pdf', 'file_path': 'data/Solar_System_Exploration_and_India_contributions.pdf', 'file_type': 'application/pdf', 'file_size': 4047124, 'creation_date': '2024-06-22', 'last_modified_date': '2024-06-05'}


In [8]:
graph_store = Neo4jGraphStore(
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    url=NEO4J_URI,
    database=NEO4J_DATABASE,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [9]:
# Clean dataset first
graph_store.query(
    """
MATCH (n) DETACH DELETE n
"""
)

[]

In [10]:
# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

In [12]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

response = query_engine.query("What is India's achievement in the latest space mission ? Give details of the Project, in few paragraphs")

In [13]:
display(Markdown(f"<b>{response}</b>"))

<b>India's achievement in the latest space mission is the successful launch and operation of the Mars Orbiter Mission, also known as Mangalyaan. Launched in November 2013, this mission marked a significant milestone for India's space exploration efforts. The Mars Orbiter Mission aimed to study the Martian atmosphere, surface, and mineral composition, as well as to explore the possibility of life on Mars.

During the mission, the Mars Orbiter spacecraft captured images of the Martian surface, reported about bright hazes in the atmosphere, and provided valuable scientific data. It carried advanced instruments like the Mars Exospheric Neutral Composition Analyser to analyze the Martian exosphere. The mission achieved a notable feat by successfully entering Mars' orbit, making India the first Asian nation to reach the Martian orbit and the fourth space agency globally to do so.

The Mars Orbiter Mission showcased India's capabilities in space technology and exploration, establishing the country as a leading force in space research. The project not only demonstrated India's technological prowess but also highlighted the country's significant contributions to solar system exploration. Dr. Tirtha Pratim Das, with his notable contributions, played a crucial role in India's success in the Mars Orbiter Mission, further solidifying India's position in the realm of space exploration.</b>

In [14]:
for node in response.source_nodes:
    print(node)

Node ID: 3ccdd9d4-3822-4079-934e-759188e9d7d2
Text: The following are knowledge sequence in max depth 2 in the form
of directed graph like: `subject -[predicate]->, object,
<-[predicate_next_hop]-, object_next_hop ...` ['BASED',
'Observations'] ['BASED', 'Observations', 'OFFER', 'Opportunity']
['BASED', 'Observations', 'OFFER', 'Advantages'] ['BASED',
'Observations', 'CAN_BE', 'Ground-based'] ['H...
Score:  1000.000



In [15]:
response = query_engine.query(
    "What is India's achievement Chandrayaan 3 as compared to previous Moon missions, compare in detail with observations",
)

display(Markdown(f"<b>{response}</b>"))

<b>India's achievement with Chandrayaan 3, in comparison to previous Moon missions, involves significant advancements in lunar exploration. Chandrayaan 3 marks a notable feat in the realm of solar system exploration, showcasing India as a leading force in space exploration. The mission builds upon the successes of previous missions like Chandrayaan 1 and the Mars Orbiter Mission, which have made remarkable strides in space exploration.

Chandrayaan 3, similar to the Mars Orbiter Mission, has conducted detailed observations and studies. It has reported about bright hazes on the Moon's surface and captured high-quality images to provide valuable scientific data. The mission carries advanced instruments like the Mars exospheric neutral composition analyser to enhance its research capabilities.

Overall, Chandrayaan 3's achievements in lunar exploration, coupled with its detailed observations and scientific contributions, demonstrate India's commitment to pushing the boundaries of space exploration and contributing significantly to our understanding of celestial bodies like the Moon.</b>

In [16]:
from llama_index.tools.neo4j import Neo4jQueryToolSpec
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent


In [17]:
llm = OpenAI(model="gpt-4", temperature=0)

gds_db = Neo4jQueryToolSpec(
    url=NEO4J_URI,
    user=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    llm=llm,
    database=NEO4J_DATABASE,
)

tools = gds_db.to_tool_list()
agent = OpenAIAgent.from_tools(tools, verbose=True)

In [18]:
agent.chat("What is India's achievement in the latest space mission ? Give details of the Project, in few paragraphs")

Added user message to memory: What is India's achievement in the latest space mission ? Give details of the Project, in few paragraphs
=== Calling Function ===
Calling function: run_request with args: {"question":"What is India's achievement in the latest space mission?","retry":true}
MATCH (e1:Entity {id: 'India'})-[:HAS_DONE]->(e2:Entity)-[:ACHIEVED]->(e3:Entity)
RETURN e3.id AS Achievement
Got output: [['Achievement']]



AgentChatResponse(response="India's achievement in the latest space mission is the successful launch of the Chandrayaan-2 mission. Chandrayaan-2 is India's second lunar exploration mission, which aimed to explore the uncharted lunar south pole region. The mission consisted of an orbiter, a lander named Vikram, and a rover named Pragyan. The primary objective of the mission was to demonstrate the ability to soft-land on the lunar surface and operate a robotic rover.\n\nThe Chandrayaan-2 mission was launched on July 22, 2019, by the Indian Space Research Organisation (ISRO) using the Geosynchronous Satellite Launch Vehicle Mark III (GSLV Mk III) rocket. The orbiter successfully entered lunar orbit on August 20, 2019, and continued to study the moon from above. However, during the landing attempt on September 7, 2019, the lander Vikram lost communication with the ground control just moments before touchdown.\n\nDespite the partial success of the mission, India's achievement in launching a

In [16]:
# Clean dataset first
graph_store.query(
    """
MATCH (n) DETACH DELETE n
"""
)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    include_embeddings=True,
)

query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)


In [19]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

response = query_engine.query("What is India's achievement in the most recent space mission ? Give details of the Project, in few paragraphs")


In [20]:
display(Markdown(f"<b>{response}</b>"))

<b>India's achievement in the most recent space mission was the successful conduct of the Mars orbiter mission. This mission, launched in November 2013, secured a notable place in the history of space exploration by providing valuable scientific data and capturing images of Mars. The Mars orbiter mission, also known as the Mangalyaan mission, showcased India's scientific prowess on a global scale. It was a significant milestone for the Indian space program, demonstrating the country's capabilities in space technology and exploration.

Additionally, India's first space-based observatory, the Aditya-l1 mission, successfully reached its desired orbit, further highlighting the country's advancements in space research. The Aditya-l1 mission serves as a testament to India's commitment to space exploration and scientific discovery. Through these missions, India has made substantial contributions to the field of space science, delving into realms of solar exploration and lunar missions.

Overall, India's recent achievements in space missions, particularly the Mars orbiter mission and the Aditya-l1 mission, have solidified its position as a leading force in space exploration. The successful outcomes of these projects have not only fueled the country's space program but have also paved the way for future endeavors in space research and technology development.</b>